![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## Person detection with additional analytics 

This notebook is an example how to use DeGirum PySDK to do person head detection with additional analytics.
The output of the person head detector is analyzed by the object tracker analyzer to associate the same person over 
the sequence of frames. Then it is analyzed by the zone analyzer to detect objects within the dedicated zone.
Then only objects within that zone are accepted for further processing. For each such object the image around
that object is cropped with some crop extent and supplied to the input of two models: age detector and gender classifier.
The results of those models are combined to obtain summary result, containing original bbox of the person's head,
it's unique track ID, cropped image of the person's head, person's age and gender.

The results are aggregated based on the person's track ID. The age/gender results are averaged over all frames on which 
that person's track ID appears to improve the accuracy of classification. Then the aggregated results are printed.

This example uses `degirum_tools.streams` streaming toolkit.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

The script can use either a web camera or local camera connected to the machine or a video file. The camera index or URL or video file path needs to be specified in the code below by assigning `video_source`.

In [9]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where do you want to run your inferences, model_zoo_url, model names for inference, and video source

In [10]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# people_det_model_name: name of the model for detecting people heads
# age_det_model_name: name of the model for age detection
# gender_det_model_name: name of the model for gender classification
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
hw_location = "@cloud"
model_zoo_url = "degirum/public"
people_det_model_name = "yolov8s_relu6_human_head--640x640_quant_n2x_orca1_1"
age_det_model_name = "yolov8n_relu6_age--256x256_quant_n2x_orca1_3"
gender_det_model_name = "mobilenetv2_050_gender--160x160_quant_n2x_orca1_1"
video_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/WalkingPeople2.mp4"

#### The rest of the cells below should run without any modifications

In [11]:
#
# Load models
#

import degirum as dg, degirum_tools
import time
from degirum_tools import streams as dgstreams

# connect to AI inference engine
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())

# load person detection model
person_det_model = zoo.load_model(
    people_det_model_name,
    output_confidence_threshold=0.7,
    overlay_show_probabilities=True,
    overlay_line_width=1,
)

# load age detection model
age_det_model = zoo.load_model(
    age_det_model_name,
    overlay_color=(255, 0, 0),
    overlay_show_probabilities=True,
)

# load gender detection model
gender_det_model = zoo.load_model(
    gender_det_model_name,
    output_top_k=1,
    overlay_color=(255, 255, 255),
    overlay_show_probabilities=False,
)

In [ ]:
#
# Create analyzers
#

# create object tracker
tracker = degirum_tools.ObjectTracker(
    class_list=[person_det_model.label_dictionary[0]],
    track_thresh=0.35,
    track_buffer=100,
    match_thresh=0.9999,
    trail_depth=20,
    anchor_point=degirum_tools.AnchorPoint.CENTER,
)

# create zone counter
counting_zone = [[[300, 10], [500, 10], [500, 710], [300, 710]]]
zone_counter = degirum_tools.ZoneCounter(
    counting_zone,
    triggering_position=degirum_tools.AnchorPoint.CENTER,
    annotation_color=(255, 0, 0),
)

# create event detector
event_name = "crossing"
event_description = f"""
    Trigger: {event_name}
    when: ZoneCount
    is greater than: 0
    during: [1, frame]
    """
event_detector = degirum_tools.EventDetector(event_description)


#
# Define cropping gizmo, which accepts only objects in counting zone
#
class InZoneCroppingGizmo(dgstreams.AiObjectDetectionCroppingGizmo):
    def validate_bbox(
        self, result: dg.postprocessor.InferenceResults, idx: int
    ) -> bool:
        """Validate detected object.

        - result: inference result
        - idx: index of object within `result.results[]` list to validate

        Returns True if object is accepted to be used for crop, False otherwise
        """
        return any(result.results[idx].get("in_zone", []))


#
# Create gizmos
#
source = dgstreams.VideoSourceGizmo(video_source)  # video source
person = dgstreams.AiSimpleGizmo(person_det_model)  # person detector
analyzer = dgstreams.AiAnalyzerGizmo(
    [tracker, zone_counter, event_detector], filters={event_name}
)  # analyzer chain with object tracker, zone counter, and event detector with filter by event
crop = InZoneCroppingGizmo(
    [person_det_model.label_dictionary[0]],
    crop_extent=15.0,
    crop_extent_option=dgstreams.CropExtentOptions.ASPECT_RATIO_ADJUSTMENT_BY_LONG_SIDE,
)  # cropping gizmo, which accepts only objects in counting zone
age = dgstreams.AiSimpleGizmo(age_det_model)  # age regression
gender = dgstreams.AiSimpleGizmo(gender_det_model)  # gender classifier
combiner = dgstreams.CropCombiningGizmo(2)  # combiner with 2 inputs
display = dgstreams.VideoDisplayGizmo(
    ["Stream"], show_ai_overlay=True, show_fps=True
)  # display
sink = dgstreams.SinkGizmo()  # result sink

#
# run composition
#
stats = {}
with dgstreams.Composition(
    source >> person >> analyzer >> crop,
    analyzer >> combiner[0] >> sink,
    crop >> age >> combiner[1],
    crop >> gender >> combiner[2],
    analyzer >> display,
):
    for data in sink():
        result = data.meta.find_last(dgstreams.tag_inference)
        frame_info = data.meta.find_last(dgstreams.tag_video)
        for obj in result.results:
            track_id = obj.get("track_id")
            extra_results = obj.get(dgstreams.CropCombiningGizmo.key_extra_results)
            if track_id is not None and extra_results:
                stat = stats.setdefault(track_id, {"scores": {}})
                for res in extra_results:
                    score_pair = stat["scores"].setdefault(
                        res.results[0]["label"], [0, 0]
                    )
                    score_pair[0] += res.results[0]["score"]
                    score_pair[1] += 1
                    stat["image"] = res.image
                    stat["frame_info"] = frame_info

#
# print statistics
#
for track_id, stat in stats.items():
    degirum_tools.ipython_display(stat["image"])
    print(
        f"ID: {track_id} ({time.ctime(stat['frame_info'][dgstreams.VideoSourceGizmo.key_timestamp])})"
    )
    for label, score_pair in stat["scores"].items():
        print(
            f"  {label}: {score_pair[0]/score_pair[1]:.2f} ({score_pair[1]} occurrences)"
        )
    print("-------------------")